In [1]:
import pandas as pd
import re

In [2]:
INPUT_HISTOLOGY_SHEET = "./resources/Histology Clearing Requests.xlsx"
input_sheet = pd.read_excel(INPUT_HISTOLOGY_SHEET, sheet_name="Histology Clearing Requests")
print(input_sheet)

                           LabTrack ID#                               Name  \
0                        753117, 753118            web-form@smartsheet.com   
1    729431 730677 732965 732966 737812            web-form@smartsheet.com   
2                                720956            web-form@smartsheet.com   
3                                711256            web-form@smartsheet.com   
4                                720935            web-form@smartsheet.com   
..                                  ...                                ...   
383                              691388  mathew.summers@alleninstitute.org   
384                              672373   kanghoon.jung@alleninstitute.org   
385                              663046   kanghoon.jung@alleninstitute.org   
386                              667574   kanghoon.jung@alleninstitute.org   
387                              673206   kanghoon.jung@alleninstitute.org   

     Requester Name              Group  \
0    Mathew Summers  

c:\Users\mae.moninghoff\AppData\Local\miniconda3\envs\aind-data-migration\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
mapping = pd.DataFrame(columns=["Requester Name", "LabTrack ID#", "Clearing Method"])

for index, row in input_sheet.iterrows():
    requester = row["Requester Name"]
    ids_string = str(row["LabTrack ID#"])
    ids_list = re.findall(r'[0-9]+', ids_string)
    ids_list = [id for id in ids_list if len(id) == 6]
    clearing_method = row["Clearing Method"]

    for id in ids_list:
        data_dict = {"Requester Name": requester, "LabTrack ID#": id, "Clearing Method": clearing_method}
        new_df = pd.DataFrame([data_dict])
        mapping = pd.concat([mapping, new_df], ignore_index=True)
    # print(requester, ids_list)
    # if requester not in mapping.keys():
    #     mapping[requester] = ids_list
    # elif requester in mapping.keys():
    #     mapping[requester].extend(ids_list)

print(mapping)

     Requester Name LabTrack ID#          Clearing Method
0    Mathew Summers       753117  LifeCanvas (Easy Index)
1    Mathew Summers       753118  LifeCanvas (Easy Index)
2     Anna Lakunina       729431  LifeCanvas (Easy Index)
3     Anna Lakunina       730677  LifeCanvas (Easy Index)
4     Anna Lakunina       732965  LifeCanvas (Easy Index)
..              ...          ...                      ...
648  Mathew Summers       691388  LifeCanvas (Easy Index)
649   Kanghoon Jung       672373  LifeCanvas (Easy Index)
650   Kanghoon Jung       663046  LifeCanvas (Easy Index)
651   Kanghoon Jung       667574  LifeCanvas (Easy Index)
652   Kanghoon Jung       673206  LifeCanvas (Easy Index)

[653 rows x 3 columns]


In [5]:
MISSING_SMARTSPIM = "./resources/missing_smartspim_info.xlsx"
TISSUE_CLEARING_PATH = "./resources/DT_HM_TissueClearingTracking_.xlsx"
missing_smartspim_df = pd.read_excel(MISSING_SMARTSPIM, sheet_name="Sheet1")

tissue_sheet_names = [ "Dec 2022 - Feb 2023", "Mar - May 2023", "Jun - Aug 2023", "Sep - Nov 2023", "Dec 2023 - Feb 2024", "Mar - May 2024", "Jun - Aug 2024"]

def load_specimen_procedure_file():
    """Load the specimen procedure file."""

    tissue_sheets = []

    for sheet_name in tissue_sheet_names:
        df = pd.read_excel(
            TISSUE_CLEARING_PATH,
            sheet_name=sheet_name,
            header=[0, 1, 2],
        )
        tissue_sheets.append(df)

    return tissue_sheets

tissue_sheets = load_specimen_procedure_file()

def is_id_in_tissue_sheet(subj_id):
    for sheet in tissue_sheets:
        if (
            int(subj_id) in sheet["SubjInfo"]["Unnamed: 0_level_1"]["Mouse ID"].tolist()
        ):
            return True
    return False


columns=["Requester", "LabTrack ID#", "Asset Name", "data_level", "Clearing Method", "on_tissue_sheet"]
tissue_presence = pd.DataFrame(columns=columns)

for index, row in missing_smartspim_df.iterrows():
    data_dict = {key: row[key] for key in columns if key in row.keys()}
    data_dict["on_tissue_sheet"] = is_id_in_tissue_sheet(data_dict["LabTrack ID#"])
    new_df = pd.DataFrame([data_dict])
    tissue_presence = pd.concat([tissue_presence, new_df], ignore_index=True)

tissue_presence.to_excel("./resources/tissue_presence.xlsx", index=False)

In [ ]:
from aind_data_access_api.document_db_ssh import DocumentDbSSHClient, DocumentDbSSHCredentials


output_df = pd.DataFrame(columns=["Requester", "LabTrack ID#", "Asset Name", "data_level", "Clearing Method",])
found_records = 0

for index, row in mapping.iterrows():
    print(f"{index}/{len(mapping)}")
    investigator = row["Requester Name"]
    id = row["LabTrack ID#"]
    clearing_method = row["Clearing Method"]

    print(f"checking for {id}")
    credentials = DocumentDbSSHCredentials()

    with DocumentDbSSHClient(credentials=credentials) as doc_db_client:
        query = {
            'name': {
                '$regex': 'smartspim', 
                '$options': 'i'
            }, 
            'data_description.subject_id': id
        }
        projection = {
            "name": 1, "created": 1, "location": 1, "data_description": 1, "last_modified": 1
        }
        document_count = doc_db_client.collection.count_documents(query)
        response = list(doc_db_client.collection.find(filter=query, projection=projection))
        print(f"found {document_count} records")
        if document_count:
            found_records += 1

    for record in response:
        asset_name = record["name"]
        data_level = record["data_description"]["data_level"]
        data_dict = {
            "Requester": investigator,
            "LabTrack ID#": id,
            "Asset Name": asset_name,
            "data_level": data_level,
            "Clearing Method": clearing_method
        }
        new_df = pd.DataFrame([data_dict])
        output_df = pd.concat([output_df, new_df], ignore_index=True)

print(output_df)
print(f"found {found_records}/{sum([len(ids) for investigators, ids in mapping.items()])} records")
output_df.to_excel("missing_smartspim_info.xlsx", index=False)


c:\Users\mae.moninghoff\AppData\Local\miniconda3\envs\aind-data-migration\Lib\site-packages\paramiko\pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\mae.moninghoff\AppData\Local\miniconda3\envs\aind-data-migration\Lib\site-packages\paramiko\transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


0/653
checking for 753117
found 0 records
1/653
checking for 753118
found 0 records
2/653
checking for 729431
found 0 records
3/653
checking for 730677
found 0 records
4/653
checking for 732965
found 0 records
5/653
checking for 732966
found 0 records
6/653
checking for 737812
found 0 records
7/653
checking for 720956
found 0 records
8/653
checking for 711256
found 0 records
9/653
checking for 720935
found 0 records
10/653
checking for 716870
found 0 records
11/653
checking for 741534
found 0 records
12/653
checking for 741535
found 0 records
13/653
checking for 741536
found 0 records
14/653
checking for 743709
found 0 records
15/653
checking for 743801
found 0 records
16/653
checking for 745297
found 0 records
17/653
checking for 736019
found 0 records
18/653
checking for 742880
found 0 records
19/653
checking for 742321
found 0 records
20/653
checking for 747772
found 0 records
21/653
checking for 737963
found 0 records
22/653
checking for 743800
found 0 records
23/653
checking for 7